In [1]:
import pandas as pd
import numpy as np
from pyomo.environ import * 

In [2]:
clients_df = pd.read_csv("Proyecto_A_Caso2/clients.csv")
depots_df = pd.read_csv("Proyecto_A_Caso2/depots.csv")
vehicles_df = pd.read_csv("Proyecto_A_Caso2/vehicles.csv")



In [3]:
print(clients_df.head())
print(depots_df.head())
print(vehicles_df.head())


   ClientID  LocationID  Product  Longitude  Latitude
0         1          13       12 -74.196992  4.632553
1         2          14       15 -74.155037  4.601328
2         3          15       15 -74.101787  4.732421
3         4          16        6 -74.194862  4.638612
4         5          17        5 -74.110272  4.727692
   DepotID  LocationID  Longitude  Latitude  Capacity
0        1           1 -74.081242  4.750212         8
1        2           2 -74.109934  4.536383        10
2        3           3 -74.038548  4.792926         0
3        4           4 -74.067069  4.721678         4
4        5           5 -74.138263  4.607707        28
  VehicleType    Capacity        Range
0     Gas Car  131.921140   145.852071
1          EV  108.435620  1304.605971
2          EV   91.504255   953.172609
3       drone   32.896064    17.302304
4       drone   22.652628    16.627680


In [4]:
C=list(depots_df["DepotID"])
CAP=dict(zip(depots_df["DepotID"],depots_df["Capacity"]))


In [5]:
print("Conjunto C:", C)
print("Capacidades CAP_i:", CAP)


Conjunto C: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
Capacidades CAP_i: {1: 8, 2: 10, 3: 0, 4: 4, 5: 28, 6: 3, 7: 0, 8: 10, 9: 43, 10: 1, 11: 16, 12: 18}


In [6]:
K=list(clients_df["ClientID"])
DEM=dict(zip(clients_df["ClientID"],clients_df["Product"]))  # Esta es la demanda que solicita ese cliente

In [7]:
print("Conjunto K: clientes", K)
print("Capacidades Demanda:", DEM)

Conjunto K: clientes [1, 2, 3, 4, 5, 6, 7, 8, 9]
Capacidades Demanda: {1: 12, 2: 15, 3: 15, 4: 6, 5: 5, 6: 11, 7: 12, 8: 10, 9: 15}


Conjunto para la distancia

In [8]:
def dist_haversiana(lon1,lat1,lon2,lat2):
  R=6371.0 #radio tierra
  lon1,lat1,lon2,lat2=map(np.radians,[lon1,lat1,lon2,lat2])
  dlon=lon2-lon1
  dlat=lat2-lat1
  a = np.sin(dlat / 2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0)**2
  c = 2 * np.arcsin(np.sqrt(a))
  return R * c



In [9]:
DIST={}
for i,depot in depots_df.iterrows():
  for j,client in clients_df.iterrows():
    di=depot["DepotID"]
    ci=client["ClientID"]
    distance=dist_haversiana(depot["Longitude"],depot["Latitude"],
                             client["Longitude"],client["Latitude"]
                             )
    DIST[(di,ci)]=round(distance,2)


In [10]:
for key, val in list(DIST.items())[:5]:
  print("La distancia entre el centro de distribucion", key[0], "y el cliente", key[1], "es de", val)

La distancia entre el centro de distribucion 1.0 y el cliente 1.0 es de 18.32
La distancia entre el centro de distribucion 1.0 y el cliente 2.0 es de 18.46
La distancia entre el centro de distribucion 1.0 y el cliente 3.0 es de 3.02
La distancia entre el centro de distribucion 1.0 y el cliente 4.0 es de 17.68
La distancia entre el centro de distribucion 1.0 y el cliente 5.0 es de 4.08


Procesamiento de vehiculos

In [11]:
vehicles_df

,VehicleType,Capacity,Range
0,Gas Car,131.921140,145.852071
1,EV,108.435620,1304.605971
2,EV,91.504255,953.172609
3,drone,32.896064,17.302304
4,drone,22.652628,16.627680
5,drone,22.682912,13.602811


In [12]:
# V=list(vehicles_df["VehicleType"])
V=list(vehicles_df.index)
Q = dict(zip(V, vehicles_df["Capacity"]))


R = dict(zip(V, vehicles_df["Range"]))



In [13]:
print("Vehículos:", V)
print("Capacidades:", Q)
print("Rangos:", R)


Vehículos: [0, 1, 2, 3, 4, 5]
Capacidades: {0: 131.9211396722696, 1: 108.4356199315333, 2: 91.50425520531196, 3: 32.896064077536955, 4: 22.65262807032524, 5: 22.682911535937688}
Rangos: {0: 145.85207096486445, 1: 1304.605971281605, 2: 953.172608610164, 3: 17.302304187458727, 4: 16.627680130757895, 5: 13.602810739289229}


In [14]:
for v in V:
    print(f"Vehículo {v}: Tipo={vehicles_df['VehicleType'][v]}, Capacidad={Q[v]}, Rango={R[v]}")

Vehículo 0: Tipo=Gas Car, Capacidad=131.9211396722696, Rango=145.85207096486445
Vehículo 1: Tipo=EV, Capacidad=108.4356199315333, Rango=1304.605971281605
Vehículo 2: Tipo=EV, Capacidad=91.50425520531196, Rango=953.172608610164
Vehículo 3: Tipo=drone, Capacidad=32.896064077536955, Rango=17.302304187458727
Vehículo 4: Tipo=drone, Capacidad=22.65262807032524, Rango=16.627680130757895
Vehículo 5: Tipo=drone, Capacidad=22.682911535937688, Rango=13.602810739289229


# Creacion del modelo

In [ ]:
model=ConcreteModel()

#Definicion de conjuntos
model.C=set(initialize=C) # Centros de distribucion

